In [17]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
import pandas as pd
import random

from itertools import product
from datetime import date
from path import Path

import os
os.environ['NUMEXPR_MAX_THREADS'] = '32'

from mlbt.run_bt import run_bt
from mlbt.feature_eng import define_feature_configs
from mlbt.load_data import LENGTH_RANKING


DATA_DIR = Path("~/Dropbox/algotrading/data").expanduser()
s_csv = pd.read_csv(DATA_DIR / "symbols.csv", index_col="iqsymbol")

# Code to execute the pipeline inside a multiprocessing.Pool
# This creates the payloads that "run_frontend_payloads.ipynb" can use to generate the
# frontend payloads (.json loaded by the frontend)
ranking = ["equity_index", ""]


symbol_groups = list(
    {
        "agriculture": "Agriculture",
        "currency": "Currency",
        "energy": "Energy",
        "equity_index": "Equity Index",
        "interest_rate": "Interest Rate",
        "metals": "Metals",
    }.keys()
)

features = define_feature_configs()
print(len(features))
c_s = (len(features) // 32) + 1 
c_s = 1
print('c_s', c_s)
data_sim = {
    "start_date": [date(2000, 1, 1)],
    "end_date": [date(2021, 1, 1)],
    "symbol_groups": [symbol_groups],
#     "symbols": [['@NQ#C']],
#     "symbols": [[x] for x in LENGTH_RANKING],
    "bar_type": ["dollar"],
    "binarize": ["fixed_horizon"],
#     "binarize_params": [6, 12],
#     "binarize_params": [6],
    "binarize_params": [6, 12, 50, 200, 400],
#     "binarize_params": [6, 25, 50],
#     "binarize_params": [200],
#     "binarize_params": [800],
    "alpha": ["none"],
#     "alpha": ["ma-cross_100_1000"],
#     "features": [features[i:i+c_s] for i in range(0, len(features), c_s)],
    "features": [features],
    "classifier": ["random_forest", "xgboost", "lgbm", "dummy"],
#     "classifier": ["xgboost"],
#     "classifier": ["lgbm"],
    "classifier": ["svc"],
#     "classifier": ["knn"],
#     "classifier": ["xgboost", "lgbm", "dummy"],
#     "classifier": ["random_forest", "xgboost", "dummy"],
#     "classifier": ["tpot"],
#     "classifier": ["lgbm"],
    "num_threads": [32], # how many parallel hyperopti 
    "n_jobs": [32], # how many parallel infer
#     "num_threads": [1], # how many parallel hyperopti 
#     "n_jobs": [1], # how many parallel infer
#     "feat_imp_n_estimators": [1000],
#     "feat_imp_cv": [10],
    "func": [run_bt],
#     "feature_calc_only": [True],
#     "feature_imp_only": [True],
#     "check_completed": [True],
#     "optimize_hypers": [False],
#     "reuse_hypers": [False],
#     "load_from_disk": [False],
    "skip_feature_imp": [True],
#     "pca_transform": [False],
#     "standard_scale": [False],
    "hypers_n_iter": [10],
}

33
c_s 1


In [21]:
datas = list(dict(zip(data_sim.keys(), x)) for x in product(*data_sim.values()))
datas
# for data in datas:
#     data['features'] = list(sorted(data['features'], key=lambda x:random.random()))


[{'start_date': datetime.date(2000, 1, 1),
  'end_date': datetime.date(2021, 1, 1),
  'symbol_groups': ['agriculture',
   'currency',
   'energy',
   'equity_index',
   'interest_rate',
   'metals'],
  'bar_type': 'dollar',
  'binarize': 'fixed_horizon',
  'binarize_params': 6,
  'alpha': 'none',
  'features': [{'name': 'log_ret'},
   {'name': 'close', 'symbol': 'VIX.XO'},
   {'name': 'ffd', 'd': 0.5},
   {'name': 'roll', 'window': 10},
   {'name': 'rollimp', 'window': 10},
   {'name': 'amihud', 'window': 10},
   {'name': 'kyle', 'window': 10},
   {'name': 'ewm_mean', 'window': 10, 'symbol': {'name': 'volratio'}},
   {'name': 'roll', 'window': 25},
   {'name': 'rollimp', 'window': 25},
   {'name': 'amihud', 'window': 25},
   {'name': 'kyle', 'window': 25},
   {'name': 'ewm_mean', 'window': 25, 'symbol': {'name': 'volratio'}},
   {'name': 'roll', 'window': 50},
   {'name': 'rollimp', 'window': 50},
   {'name': 'amihud', 'window': 50},
   {'name': 'kyle', 'window': 50},
   {'name': 'ewm_

In [22]:
len(datas)

5

In [23]:
# [x['features'][0] for x in datas]

In [24]:
from mlbt.multiprocess import process_jobs

In [25]:
num_threads = max(1, min(32, len(datas)) - 0)
num_threads
      

5

In [16]:
res = process_jobs(datas, num_threads=num_threads)

2020-02-23 01:37:43.823585 100.0% run_bt done after 4.78 minutes. Remaining 0.0 minutes..


In [ ]:
for data in datas:
    run_bt(**data)

2020-02-23 01:55:06,207 config: {'DATA_DIR': Path('C:\\Users\\doda/Dropbox/algotrading/data'), 'F_PAYLOAD_DIR': Path('C:\\Users\\doda/pr/fincl/frontend/public/payloads'), 'start_date': datetime.date(2000, 1, 1), 'end_date': datetime.date(2021, 1, 1), 'data_freq': 'minutely', 'downsampling': 'cusum', 'vol_estimate': 100, 'symbol_groups': ['agriculture', 'currency', 'energy', 'equity_index', 'interest_rate', 'metals'], 'features': [{'name': 'log_ret'}, {'name': 'close', 'symbol': 'VIX.XO'}, {'name': 'ffd', 'd': 0.5}, {'name': 'roll', 'window': 10}, {'name': 'rollimp', 'window': 10}, {'name': 'amihud', 'window': 10}, {'name': 'kyle', 'window': 10}, {'name': 'ewm_mean', 'window': 10, 'symbol': {'name': 'volratio'}}, {'name': 'roll', 'window': 25}, {'name': 'rollimp', 'window': 25}, {'name': 'amihud', 'window': 25}, {'name': 'kyle', 'window': 25}, {'name': 'ewm_mean', 'window': 25, 'symbol': {'name': 'volratio'}}, {'name': 'roll', 'window': 50}, {'name': 'rollimp', 'window': 50}, {'name': '

2020-02-23 01:55:12,147 XG#C: Feature engineering for 33 features
2020-02-23 01:55:12,487 Joined 33 features into (71217, 33) shape
2020-02-23 01:55:12,487 EX#C: Feature engineering for 33 features
2020-02-23 01:55:12,827 Joined 33 features into (74018, 33) shape
2020-02-23 01:55:12,828 @YM#C: Feature engineering for 33 features
2020-02-23 01:55:13,150 Joined 33 features into (39311, 33) shape
2020-02-23 01:55:13,152 @BO#C: Feature engineering for 33 features
2020-02-23 01:55:13,503 Joined 33 features into (93715, 33) shape
2020-02-23 01:55:13,504 @C#C: Feature engineering for 33 features
2020-02-23 01:55:13,846 Joined 33 features into (80542, 33) shape
2020-02-23 01:55:13,848 @FV#C: Feature engineering for 33 features
2020-02-23 01:55:14,177 Joined 33 features into (53497, 33) shape
2020-02-23 01:55:14,178 @TY#C: Feature engineering for 33 features
2020-02-23 01:55:14,507 Joined 33 features into (55083, 33) shape
2020-02-23 01:55:14,508 @TU#C: Feature engineering for 33 features
2020-

2020-02-23 01:55:25,716 BD#C: Get bins and feature imps
2020-02-23 01:55:25,731 Dropped label 0.0 0.012826514371313548
2020-02-23 01:55:25,948 bars (68392, 9), events (32049, 2), feats (68392, 33), bins (31632, 3), X_all (29244, 33), X_train (14622, 33)
2020-02-23 01:55:25,949 EZ#C: Get bins and feature imps
2020-02-23 01:55:25,967 Dropped label 0.0 0.05257124140189977
2020-02-23 01:55:26,203 bars (110422, 9), events (42748, 2), feats (110422, 33), bins (40495, 3), X_all (35327, 33), X_train (17663, 33)
2020-02-23 01:55:26,203 XG#C: Get bins and feature imps
2020-02-23 01:55:26,221 Dropped label 0.0 0.0038793353019512477
2020-02-23 01:55:26,449 bars (71217, 9), events (34548, 2), feats (71217, 33), bins (34408, 3), X_all (31656, 33), X_train (15828, 33)
2020-02-23 01:55:26,450 EX#C: Get bins and feature imps
2020-02-23 01:55:26,466 Dropped label 0.0 0.01553488102483842
2020-02-23 01:55:26,693 bars (74018, 9), events (34509, 2), feats (74018, 33), bins (33967, 3), X_all (31551, 33), X_t

2020-02-23 01:55:33,080 Dropped label 0.0 0.0035895901884534847
2020-02-23 01:55:33,263 bars (67132, 9), events (10035, 2), feats (67132, 33), bins (9993, 3), X_all (9649, 33), X_train (4824, 33)
2020-02-23 01:55:33,264 QSI#C: Get bins and feature imps
2020-02-23 01:55:33,278 Dropped label 0.0 0.007809387202167503
2020-02-23 01:55:33,482 bars (59640, 9), events (25104, 2), feats (59640, 33), bins (24902, 3), X_all (24467, 33), X_train (12233, 33)
2020-02-23 01:55:33,485 @AD#C [(905, 5), (905, 33), (905,), (905, 5), (905, 33), (905,)]
2020-02-23 01:55:33,485 @BP#C [(2534, 5), (2534, 33), (2534,), (2535, 5), (2535, 33), (2535,)]
2020-02-23 01:55:33,486 @CD#C [(1144, 5), (1144, 33), (1144,), (1145, 5), (1145, 33), (1145,)]
2020-02-23 01:55:33,486 @ED#C [(6441, 5), (6441, 33), (6441,), (6442, 5), (6442, 33), (6442,)]
2020-02-23 01:55:33,486 @ES#C [(12888, 5), (12888, 33), (12888,), (12888, 5), (12888, 33), (12888,)]
2020-02-23 01:55:33,487 @NQ#C [(7135, 5), (7135, 33), (7135,), (7135, 5), 

In [10]:
%debug

> c:\users\doda\pr\fincl\backend\mlbt\run_bt.py(165)train_test_split()
    163     X_train, X_test = X_all.loc[:cut], X_all.loc[cut:]
    164     y_train, y_test = y_all.loc[:cut], y_all.loc[cut:]
--> 165     assert X_train.shape[0] > 0
    166 
    167     logging.info(f"bars {bars.shape}, events {events.shape}, feats {feats.shape}, bins {bins.shape}, X_all {X_all.shape}, X_train {X_train.shape}")

ipdb> X_all.index
DatetimeIndex(['2015-02-04 14:14:00', '2015-02-04 14:25:00',
               '2015-02-05 02:48:00', '2015-02-05 03:57:00',
               '2015-02-05 05:10:00', '2015-02-05 06:26:00',
               '2015-02-05 06:56:00', '2015-02-05 08:02:00',
               '2015-02-05 09:14:00', '2015-02-05 10:02:00',
               ...
               '2019-10-09 10:13:00', '2019-10-09 11:01:00',
               '2019-10-09 11:58:00', '2019-10-09 13:03:00',
               '2019-10-09 13:45:00', '2019-10-09 14:29:00',
               '2019-10-09 16:19:00', '2019-10-10 03:42:00',
           